# This notebook cleans the data extracted from the Second Chamber API

In this file, the data is selected, cleaned and made ready to directly implement in the IBM Cloud functions. 
The final result is a new JSON file.

In [57]:
#importing the libraries
import pandas as pd #to read pandas
import json #to read the JSON file

In [58]:
motions_data = pd.read_json('https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document?$filter=Soort%20eq%20%27Motie%27%20and%20contains(Onderwerp,%27corona%27)', orient='columns') #importing the JSON file 
motions_all = pd.read_json((motions_data['value']).to_json(), orient='index') #and converting it to a dataframe

In [65]:
#selecting the relavent columns from the original file
#the selection is based on the feedback received from the user tests

motions = motions_all[['Titel','Volgnummer','Soort','Onderwerp','Datum']] #selecting the columns

motions.head(5)

,Titel,Volgnummer,Soort,Onderwerp,Datum
0,Wijziging van de begrotingsstaten van het Mini...,9,Motie,Motie van het lid Amhaouch c.s. over steun voo...,2020-05-18T00:00:00+02:00
1,Infectieziektenbestrijding,357,Motie,Motie van de leden Jetten en Pieter Heerma ove...,2020-05-20T00:00:00+02:00
10,Wijziging van de begrotingsstaten van het Mini...,4,Motie,Motie van het lid Graus c.s. over een gelijk s...,2020-05-18T00:00:00+02:00
11,Infectieziektenbestrijding,253,Motie,Motie van het lid Wilders c.s. over zorgmedewe...,2020-04-16T00:00:00+02:00
12,Infectieziektenbestrijding,344,Motie,Motie van de leden Van Haga en Hiddema over ge...,2020-05-07T00:00:00+02:00


<h2>Note</h2>
<br>
As seen above, some of the data is not readable e.g, date. <br>Also, for the IBM implementation, additional columns are required such as 'applicants' and 'theme'.

In [66]:
#adding the column 'Indiender' (applicants)
motions = motions[motions['Onderwerp'].notnull()].copy() #adding a new column to display the applicant, this will be extracted from 'onderwerp'
motions['Indieners'] = motions['Onderwerp'].astype(str) #converted into strings
motions['Indieners'] = motions['Indieners'].map(lambda x: x.lstrip('Motie van het lid').rstrip('aAbBcC')) #remove text
motions['Indieners'] = motions['Indieners'].str.split('c.s').str[0] #remove text
motions['Indieners'] = motions['Indieners'].str.split('over').str[0] #remove text


motions.head(5)

,Titel,Volgnummer,Soort,Onderwerp,Datum,Indieners
0,Wijziging van de begrotingsstaten van het Mini...,9,Motie,Motie van het lid Amhaouch c.s. over steun voo...,2020-05-18T00:00:00+02:00,Amhaouch
1,Infectieziektenbestrijding,357,Motie,Motie van de leden Jetten en Pieter Heerma ove...,2020-05-20T00:00:00+02:00,Jetten en Pieter Heerma
10,Wijziging van de begrotingsstaten van het Mini...,4,Motie,Motie van het lid Graus c.s. over een gelijk s...,2020-05-18T00:00:00+02:00,Graus
11,Infectieziektenbestrijding,253,Motie,Motie van het lid Wilders c.s. over zorgmedewe...,2020-04-16T00:00:00+02:00,Wilders
12,Infectieziektenbestrijding,344,Motie,Motie van de leden Van Haga en Hiddema over ge...,2020-05-07T00:00:00+02:00,Van Haga en Hiddema


In [67]:
#transforming the 'date'
import datetime
import locale #to translate the date
locale.setlocale(locale.LC_TIME, "nl_NL") #set to Dutch


motions['Datum'] = motions['Datum'].str.split('T').str[0] #removing the time
motions['Datum'] = pd.to_datetime(motions.Datum) #converting the string to Date
motions = motions.sort_values(by='Datum', ascending=False) #moste recent motions first
#motions['Datum'] = motions['Datum'].dt.strftime('%Y-%b-%d') #transforms the Date back to string
motions['Datum'] = motions['Datum'].dt.strftime('%d %B, %Y') #transforms the Date back to string, and more makes it readable

motions.head(5)

,Titel,Volgnummer,Soort,Onderwerp,Datum,Indieners
24,Infectieziektenbestrijding,390,Motie,Motie van de leden Wilders en Agema over de aa...,"04 juni, 2020",Wilders en Agema
52,Infectieziektenbestrijding,392,Motie,Motie van het lid Veldman c.s. over nut en noo...,"04 juni, 2020",Veldman
3,Infectieziektenbestrijding,407,Motie,Motie van het lid Van Brenk over herzien van h...,"04 juni, 2020",Van Brenk
25,Infectieziektenbestrijding,399,Motie,Motie van de leden Asscher en Marijnissen over...,"04 juni, 2020",Asscher en Marijnissen
22,Infectieziektenbestrijding,409,Motie,Motie van het lid Kuzu over mogelijkheden voor...,"04 juni, 2020",Kuzu


<h2>Note</h2>

Some of the data is missing because it could not be extracted from the API, due to lack of knowlegde. However, the columns were added to provide a preview of the research's intention. The selection is based on the feedback received from the user tests.

In [68]:
motions['Status'] = 'tegen/voor gestemd'  #voting behaviour
motions['Inhoud'] = 'meer informatie' #additional informtion 
motions = motions.rename(columns = {'Titel':'Thema'}) #theme's were created to provide an overview
motions['Thema'] = motions['Thema'].replace({'Noodpakket banen en economie':'Economie',
                                             'Financieel jaarverslag van het Rijk 2019':'Financieel',
                                            'Raad Algemene Zaken en Raad Buitenlandse Zaken':'Buitenlandse-Zaken',
                                            'Vaste commissie voor Economische Zaken en Klimaat':'Klimaat'})

In [69]:
motions

,Thema,Volgnummer,Soort,Onderwerp,Datum,Indieners,Status,Inhoud
24,Infectieziektenbestrijding,390,Motie,Motie van de leden Wilders en Agema over de aa...,"04 juni, 2020",Wilders en Agema,tegen/voor gestemd,meer informatie
52,Infectieziektenbestrijding,392,Motie,Motie van het lid Veldman c.s. over nut en noo...,"04 juni, 2020",Veldman,tegen/voor gestemd,meer informatie
3,Infectieziektenbestrijding,407,Motie,Motie van het lid Van Brenk over herzien van h...,"04 juni, 2020",Van Brenk,tegen/voor gestemd,meer informatie
25,Infectieziektenbestrijding,399,Motie,Motie van de leden Asscher en Marijnissen over...,"04 juni, 2020",Asscher en Marijnissen,tegen/voor gestemd,meer informatie
22,Infectieziektenbestrijding,409,Motie,Motie van het lid Kuzu over mogelijkheden voor...,"04 juni, 2020",Kuzu,tegen/voor gestemd,meer informatie
21,Infectieziektenbestrijding,400,Motie,Motie van het lid Asscher over aan te passen b...,"04 juni, 2020",Asscher,tegen/voor gestemd,meer informatie
40,Vreemdelingenbeleid,2625,Motie,Motie van de leden Voordewind en Groothuizen o...,"03 juni, 2020",Voordewind en Groothuizen,tegen/voor gestemd,meer informatie
4,Economie,61,Motie,Motie van de leden Van Haga en Baudet over de ...,"28 mei, 2020",Van Haga en Baudet,tegen/voor gestemd,meer informatie
17,Economie,57,Motie,Motie van het lid Pieter Heerma c.s. over een ...,"28 mei, 2020",Pieter Heerma,tegen/voor gestemd,meer informatie
49,Financieel,15,Motie,Motie van het lid Kuzu over aandacht voor doel...,"26 mei, 2020",Kuzu,tegen/voor gestemd,meer informatie


In [71]:
#Due to lack of access and knowlegde to the API, additional data was typed over from the Second Chamber website. 
#Covid-19 bills were added
motions.loc[motions.index.max()+1] = ['Vaste commissie voor Economische Zaken en Klimaat',
                                      '982',
                                      'Wetsvoorstel',
                                      'Tijdelijke wet informatieverstrekking RIVM in verband met COVID-19',
                                      '29 mei, 2020',
                                      'M.C.G. Keijzer',
                                      'Voorbereiding',
                                      'Dit wetsvoorstel creëert een grondslag in hoofdstuk 14 van de Telecommunicatiewet om aanbieders van openbare mobiele telecommunicatienetwerken te verplichten informatie te verstrekken aan het Rijksinstituut Rijksinstituut voor volksgezondheid en milieu (RIVM) ter ondersteuning bij de bestrijding van de het coronavirus. Het is voor het RIVM noodzakelijk om drukte en verplaatsingen op bevolkingsniveau te kunnen monitoren. Het gaat nadrukkelijk niet om het volgen maar om het tellen van personen.']
motions

,Thema,Volgnummer,Soort,Onderwerp,Datum,Indieners,Status,Inhoud
24,Infectieziektenbestrijding,390,Motie,Motie van de leden Wilders en Agema over de aa...,"04 juni, 2020",Wilders en Agema,tegen/voor gestemd,meer informatie
52,Infectieziektenbestrijding,392,Motie,Motie van het lid Veldman c.s. over nut en noo...,"04 juni, 2020",Veldman,tegen/voor gestemd,meer informatie
3,Infectieziektenbestrijding,407,Motie,Motie van het lid Van Brenk over herzien van h...,"04 juni, 2020",Van Brenk,tegen/voor gestemd,meer informatie
25,Infectieziektenbestrijding,399,Motie,Motie van de leden Asscher en Marijnissen over...,"04 juni, 2020",Asscher en Marijnissen,tegen/voor gestemd,meer informatie
22,Infectieziektenbestrijding,409,Motie,Motie van het lid Kuzu over mogelijkheden voor...,"04 juni, 2020",Kuzu,tegen/voor gestemd,meer informatie
21,Infectieziektenbestrijding,400,Motie,Motie van het lid Asscher over aan te passen b...,"04 juni, 2020",Asscher,tegen/voor gestemd,meer informatie
40,Vreemdelingenbeleid,2625,Motie,Motie van de leden Voordewind en Groothuizen o...,"03 juni, 2020",Voordewind en Groothuizen,tegen/voor gestemd,meer informatie
4,Economie,61,Motie,Motie van de leden Van Haga en Baudet over de ...,"28 mei, 2020",Van Haga en Baudet,tegen/voor gestemd,meer informatie
17,Economie,57,Motie,Motie van het lid Pieter Heerma c.s. over een ...,"28 mei, 2020",Pieter Heerma,tegen/voor gestemd,meer informatie
49,Financieel,15,Motie,Motie van het lid Kuzu over aandacht voor doel...,"26 mei, 2020",Kuzu,tegen/voor gestemd,meer informatie


In [72]:
#converting the datafrome into a new JSON file

motions.to_json(r'jsons/moties-wetsvoorstellen.json', orient='table') 